In [19]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/libs

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/libs


In [20]:
%matplotlib inline
import numpy as np
import scipy as sp
import scipy.sparse as sps
import matplotlib.pyplot as plt
import random
from scipy import stats
from scipy.optimize import fmin
import pandas as pd

In [21]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

data = pd.read_csv(filepath_or_buffer="data_train.csv",
                   sep=",")
data.columns = ["UserID", "ItemID", "Interaction"]

mapped_id, original_id = pd.factorize(data["UserID"].unique())
user_original_ID_to_index = pd.Series(mapped_id, index=original_id)

mapped_id, original_id = pd.factorize(data["ItemID"].unique())
item_original_ID_to_index = pd.Series(mapped_id, index=original_id)

data["UserID"] = data["UserID"].map(user_original_ID_to_index)
data["ItemID"] = data["ItemID"].map(item_original_ID_to_index)


URM_all = sps.coo_matrix((data["Interaction"].values,
                          (data["UserID"].values, data["ItemID"].values)))

URM_all.tocsr()

URM_split_1, URM_fold_1 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_split_2, URM_split_3 = split_train_in_two_percentage_global_sample(URM_split_1, train_percentage = 0.5)
URM_fold_2, URM_fold_3 = split_train_in_two_percentage_global_sample(URM_split_2, train_percentage = 0.5)
URM_fold_4, URM_fold_5 = split_train_in_two_percentage_global_sample(URM_split_3, train_percentage = 0.5)

URM_all, URM_fold_1, URM_fold_2, URM_fold_3, URM_fold_4, URM_fold_5

(<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
 	with 478730 stored elements in COOrdinate format>,
 <12638x22222 sparse matrix of type '<class 'numpy.float64'>'
 	with 95746 stored elements in Compressed Sparse Row format>,
 <12638x22222 sparse matrix of type '<class 'numpy.float64'>'
 	with 95746 stored elements in Compressed Sparse Row format>,
 <12638x22222 sparse matrix of type '<class 'numpy.float64'>'
 	with 95746 stored elements in Compressed Sparse Row format>,
 <12638x22222 sparse matrix of type '<class 'numpy.float64'>'
 	with 95746 stored elements in Compressed Sparse Row format>,
 <12638x22222 sparse matrix of type '<class 'numpy.float64'>'
 	with 95746 stored elements in Compressed Sparse Row format>)

#Try Hyperparameter tuning with Optuna

In [22]:
 pip install optuna

In [23]:
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender



evaluator_1 = EvaluatorHoldout(URM_fold_1, cutoff_list=[10])
evaluator_2 = EvaluatorHoldout(URM_fold_2, cutoff_list=[10])
evaluator_3 = EvaluatorHoldout(URM_fold_3, cutoff_list=[10])
evaluator_4 = EvaluatorHoldout(URM_fold_4, cutoff_list=[10])
evaluator_5 = EvaluatorHoldout(URM_fold_5, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2177 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2159 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2145 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2153 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2196 (17.4%) Users that have less than 1 test interactions


In [24]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

rp = RP3betaRecommender(URM_fold_1 + URM_fold_4 + URM_fold_3 + URM_fold_5)
rp.fit(topK = 95, alpha = 0.49, beta = 0.20)
res, _ = evaluator_2.evaluateRecommender(rp)
res

RP3betaRecommender: URM Detected 218 ( 1.7%) users with no interactions.
RP3betaRecommender: URM Detected 104 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 2266.02 column/sec. Elapsed time 9.81 sec
EvaluatorHoldout: Processed 10479 (100.0%) in 7.45 sec. Users per second: 1407


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.092232,0.16196,0.137118,0.046416,0.081349,0.261286,0.146656,0.110283,0.513026,0.343711,...,0.829166,0.425384,0.829166,0.070819,10.133721,0.996471,0.203248,0.780664,2.013666,0.288264


In [25]:
res.loc[10]["MAP"]

0.046416165664663236

In [26]:
URM_temp_1 = URM_fold_1 + URM_fold_2 + URM_fold_3 + URM_fold_4
URM_temp_2 = URM_fold_1 + URM_fold_2 + URM_fold_3 + URM_fold_5
URM_temp_3 = URM_fold_1 + URM_fold_2 + URM_fold_4 + URM_fold_5
URM_temp_4 = URM_fold_1 + URM_fold_3 + URM_fold_4 + URM_fold_5
URM_temp_5 = URM_fold_2 + URM_fold_3 + URM_fold_4 + URM_fold_5

URM_temp_1, URM_temp_2, URM_temp_3, URM_temp_4, URM_temp_5

(<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
 	with 382984 stored elements in Compressed Sparse Row format>,
 <12638x22222 sparse matrix of type '<class 'numpy.float64'>'
 	with 382984 stored elements in Compressed Sparse Row format>,
 <12638x22222 sparse matrix of type '<class 'numpy.float64'>'
 	with 382984 stored elements in Compressed Sparse Row format>,
 <12638x22222 sparse matrix of type '<class 'numpy.float64'>'
 	with 382984 stored elements in Compressed Sparse Row format>,
 <12638x22222 sparse matrix of type '<class 'numpy.float64'>'
 	with 382984 stored elements in Compressed Sparse Row format>)

In [27]:
import optuna
import pandas as pd
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
#"alpha": Real(0.1,0.2),    "beta" : Real(0.1,0.2),    "topK" : Integer(10,50)

def objective_function(optuna_trial):



    recommender_instance1 = SLIMElasticNetRecommender(URM_fold_1 + URM_fold_2 + URM_fold_3 + URM_fold_4)
    recommender_instance1.fit(topK = optuna_trial.suggest_int("topK", 200, 300),
                             alpha = optuna_trial.suggest_float("alpha", 0.001, 0.1),
                             l1_ratio = optuna_trial.suggest_float("l1_ratio", 1e-6, 1e-4)
                            )
    r1, _ = evaluator_5.evaluateRecommender(recommender_instance1)

    print(r1.loc[10]["MAP"])


    recommender_instance2 = SLIMElasticNetRecommender(URM_fold_1 + URM_fold_2 + URM_fold_3 + URM_fold_5)
    recommender_instance2.fit(topK = optuna_trial.suggest_int("topK", 200, 300),
                             alpha = optuna_trial.suggest_float("alpha", 0.001, 0.1),
                             l1_ratio = optuna_trial.suggest_float("l1_ratio", 1e-6, 1e-4)
                            )
    r2, _ = evaluator_4.evaluateRecommender(recommender_instance2)

    print(r2.loc[10]["MAP"])


    recommender_instance3 = SLIMElasticNetRecommender(URM_fold_1 + URM_fold_2 + URM_fold_4 + URM_fold_5)
    recommender_instance3.fit(topK = optuna_trial.suggest_int("topK", 200, 300),
                             alpha = optuna_trial.suggest_float("alpha", 0.001, 0.1),
                             l1_ratio = optuna_trial.suggest_float("l1_ratio", 1e-6, 1e-4)
                            )
    r3, _ = evaluator_3.evaluateRecommender(recommender_instance3)

    print(r3.loc[10]["MAP"])


    recommender_instance4 = SLIMElasticNetRecommender(URM_fold_1 + URM_fold_3 + URM_fold_4 + URM_fold_5)
    recommender_instance4.fit(topK = optuna_trial.suggest_int("topK", 200, 300),
                             alpha = optuna_trial.suggest_float("alpha", 0.001, 0.1),
                             l1_ratio = optuna_trial.suggest_float("l1_ratio", 1e-6, 1e-4)
                            )
    r4, _ = evaluator_2.evaluateRecommender(recommender_instance4)

    print(r4.loc[10]["MAP"])

    recommender_instance5 = SLIMElasticNetRecommender(URM_fold_2 + URM_fold_3 + URM_fold_4 + URM_fold_5)
    recommender_instance5.fit(topK = optuna_trial.suggest_int("topK", 200, 300),
                             alpha = optuna_trial.suggest_float("alpha", 0.001, 0.1),
                             l1_ratio = optuna_trial.suggest_float("l1_ratio", 1e-6, 1e-4)
                            )
    r5, _ = evaluator_1.evaluateRecommender(recommender_instance5)

    print(r5.loc[10]["MAP"])

    res = (r1.loc[10]["MAP"] + r2.loc[10]["MAP"] + r3.loc[10]["MAP"] + r4.loc[10]["MAP"] + r5.loc[10]["MAP"])/5

    return res

In [28]:
class SaveResults(object):

    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])

    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]

        self.results_df = self.results_df.append(hyperparam_dict, ignore_index=True)

In [ ]:
optuna_study = optuna.create_study(direction="maximize")

save_results = SaveResults()

optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100)

[I 2023-12-24 17:41:04,987] A new study created in memory with name: no-name-fa4abad4-71e4-4133-bd0a-05fadab04ebc


SLIMElasticNetRecommender: URM Detected 200 ( 1.6%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 117 ( 0.5%) items with no interactions.


In [ ]:
pruned_trials = [t for t in optuna_study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in optuna_study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(optuna_study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
print("  Value Validation: ", optuna_study.best_trial.value)

In [ ]:
optuna_study.best_trial

In [ ]:
optuna_study.best_trial.params

In [ ]:
save_results.results_df

In [ ]:
recommender_instance = RP3betaRecommender(URM_train + URM_validation)
recommender_instance.fit(**optuna_study.best_trial.params)

result_df, _ = evaluator_test.evaluateRecommender(recommender_instance)
result_df

#Let's submit

In [5]:
#{'alpha': 0.4212426697582987, 'beta': 0.2400809967920454, 'topK': 65, 'implicit': False, 'normalize_similarity': True} val:2.97% test:3.7% submit: 12.368%

#{'alpha': 0.1799686385529828, 'beta': 0.12976889396960958, 'topK': 33, 'implicit': False, 'normalize_similarity': True} val:2.99% test:3.752% submit:12.956%

#{'alpha': 0.3, 'beta': 0.1776976699523311, 'topK': 31, 'implicit': True, 'normalize_similarity': True} val:3.044% test: 3.758% submit:12.946%

#{'alpha': 0.1746971072522342, 'beta': 0.15708290766128252, 'topK': 38, 'implicit': False, 'normalize_similarity': True} val3.007%: test:3.779%

#{'alpha': 0.2, 'beta': 0.1, 'topK': 50} val:3.018% test:3.782%

#{'alpha': 0.16867631478622236, 'beta': 0.1033407840075039, 'topK': 32} val:2.989% test:3.806% submit:12.896% con nuovo topPop

#OPTUNA

#{'topK': 33, 'alpha': 0.403715416892729, 'beta': 0.11470570616457175} val:3.046% test(dopo ritrain su train e val): 4.832%

#{'topK': 31, 'alpha': 0.40358163730763214, 'beta': 0.14844292552666044} val:3.114% test(train+val):4.827%

#SENZA TEST, CON TRAINING USANDO TRAIN_VAL E VALIDATION USANDO TEST

#CV: {'topK': 29, 'alpha': 0.31583032356728813, 'beta': 0.1936831182299878} val: 4.9014%



from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

recommender = RP3betaRecommender(URM_all)
recommender.fit(alpha = 0.31583032356728813, beta=0.1936831182299878, topK=29, implicit = False, normalize_similarity = True )

RP3betaRecommender: Similarity column 22222 (100.0%), 3861.08 column/sec. Elapsed time 5.76 sec


In [8]:
users_to_recommend_raw = pd.read_csv(filepath_or_buffer="/gdrive/My Drive/libs/data_target_users_test.csv",
                   sep=",",
                  dtype={"user_id": np.int32}).to_numpy()
users_to_recommend = []
for elem in users_to_recommend_raw:
  users_to_recommend.append(elem[0])

len(users_to_recommend)

10882

In [9]:
ratings = pd.read_csv("/gdrive/My Drive/libs/data_train2.csv",
                       sep=",",
                       names=["user_id", "item_id", "ratings"],
                       header=None,
                       dtype={"user_id": np.int64,
                               "item_id": np.int64,
                               "ratings": np.int64})
ratings.shape

(478730, 3)

In [10]:
def preprocess_data(ratings: pd.DataFrame):
    unique_users = ratings.user_id.unique()
    unique_items = ratings.item_id.unique()

    num_users, min_user_id, max_user_id = unique_users.size, unique_users.min(), unique_users.max()
    num_items, min_item_id, max_item_id = unique_items.size, unique_items.min(), unique_items.max()

    print(num_users, min_user_id, max_user_id)
    print(num_items, min_item_id, max_item_id)

    mapping_user_id = pd.DataFrame({"mapped_user_id": np.arange(num_users), "user_id": unique_users})
    mapping_item_id = pd.DataFrame({"mapped_item_id": np.arange(num_items), "item_id": unique_items})

    ratings = pd.merge(left=ratings,
                       right=mapping_user_id,
                       how="inner",
                       on="user_id")

    ratings = pd.merge(left=ratings,
                       right=mapping_item_id,
                       how="inner",
                       on="item_id")

    return ratings


In [11]:
ratings = preprocess_data(ratings)

12638 1 13024
22222 1 22347


In [12]:
users_ids_and_mappings = ratings[ratings.user_id.isin(users_to_recommend)][["user_id", "mapped_user_id"]].drop_duplicates()
items_ids_and_mappings = ratings[["item_id", "mapped_item_id"]].drop_duplicates()
users_ids_and_mappings

,user_id,mapped_user_id
0,1,0
1,2,1
2,26,24
3,36,34
4,41,39
...,...,...
469974,9966,9677
470088,11525,11196
471697,11994,11649
476929,10065,9773


In [13]:
def prepare_submission(ratings: pd.DataFrame, users_to_recommend: np.array, urm_train: sps.csr_matrix):
    users_ids_and_mappings = ratings[ratings.user_id.isin(users_to_recommend)][["user_id", "mapped_user_id"]].drop_duplicates()
    items_ids_and_mappings = ratings[["item_id", "mapped_item_id"]].drop_duplicates()

    mapping_to_item_id = dict(zip(ratings.mapped_item_id, ratings.item_id))


    recommendation_length = 10
    submission = []
    for idx, row in users_ids_and_mappings.iterrows():
        user_id = row.user_id
        mapped_user_id = row.mapped_user_id
        recommendations = recommender.recommend(mapped_user_id,recommendation_length)
        submission.append((user_id, [mapping_to_item_id[item_id] for item_id in recommendations]))

    users_recommended = [elem[0] for elem in submission]

    missing_users = []

    top10 = [2, 4, 1, 7, 3, 6, 8, 9, 15, 20]  #con remove seen = False
    for elem in users_to_recommend:
      if elem not in users_recommended:
        missing_users.append(elem)

    for i in range(len(missing_users)):
      missing_users[i] = (missing_users[i], [mapping_to_item_id[top10[elem]] for elem in range(10)])

    return submission, missing_users

In [14]:
URM_train = URM_all.tocsr()
submission, missing_users = prepare_submission(ratings, users_to_recommend, URM_all)
len(submission), len(missing_users)


(10661, 221)

In [15]:
submission = submission + missing_users
len(submission)

10882

In [16]:
submission.sort()

In [17]:
def write_submission(submissions):
    with open("./RP3Beta_CV.csv", "w") as f:
        for user_id, items in submissions:
          f.write(f"{user_id},{' '.join(str(item) for item in items)}\n")



In [18]:
write_submission(submission)